# Caso de estudio 1:

Este código realiza la optimización del volumen de una caja mientras respeta las restricciones de superficie, utilizando un algoritmo genético que involucra selección, cruce, mutación y reemplazo elitista.

In [ ]:
import numpy as np

## 1. **Generación de la Población Inicial**
La función `generar_poblacion(n, l)` crea una población inicial de cromosomas binarios, donde:
- `n` es el número de individuos en la población.
- `l` es la longitud de cada cromosoma (en este caso, 9 bits para representar las tres dimensiones de la caja).

In [ ]:
def generar_poblacion(n, l):
    return np.random.randint(2, size=(n, l))

poblacion = generar_poblacion(5, 9)
print(poblacion)

[[1 1 0 1 1 1 1 0 0]
 [1 1 1 1 0 1 1 1 1]
 [1 1 0 0 0 0 1 0 1]
 [1 0 0 1 0 1 0 1 1]
 [0 1 1 0 0 1 0 0 1]]


## 2. **Decodificación de Cromosomas**
Cada cromosoma binario es decodificado en valores decimales que representan las dimensiones `l` (longitud), `w` (ancho), y `h` (altura) de la caja. La función `bin2dec` convierte un segmento binario en un valor decimal dentro de un rango definido, y `decodificar_cromosoma` aplica esta conversión a las tres dimensiones.

In [ ]:
def bin2dec(cadena, _min, _max):
    longitud = len(cadena)
    valor_posicional = np.asarray([2**i for i in range(longitud)])[::-1]
    valor_decimal = cadena.dot(valor_posicional)
    max_binario = (2 ** longitud) - 1
    return _min + (valor_decimal / max_binario) * (_max - _min)

def decodificar_cromosoma(cromosoma, _min, _max):
    longitud = len(cromosoma) // 3
    l = bin2dec(cromosoma[:longitud], _min, _max)
    w = bin2dec(cromosoma[longitud:2*longitud], _min, _max)
    h = bin2dec(cromosoma[2*longitud:], _min, _max)
    return np.array([l, w, h])


## 3. **Función de Aptitud**
La aptitud de un cromosoma se calcula como el volumen de la caja, pero con una penalización si el área superficial excede un límite (20 en este caso).

In [ ]:
def funcion_aptitud(l, w, h):
    volumen = l * w * h
    area = 2 * (l * w + l * h + w * h)
    if area > 20:
        return 0  # Penalización si no se cumple la restricción
    return volumen


## 4. **Selección por Torneo**
La función `seleccion_torneo` selecciona a un individuo para la reproducción mediante un proceso de torneo. Se elige un subconjunto aleatorio de la población y se selecciona el individuo con la mayor aptitud.

In [ ]:
def seleccion_torneo(poblacion, k):
    N = len(poblacion)
    idx_torneo = np.random.choice(N, k)
    torneo = poblacion[idx_torneo]
    aptitudes = np.array([funcion_aptitud(*decodificar_cromosoma(ind, 0, 5)) for ind in torneo])
    ganador = torneo[np.argmax(aptitudes)]
    return ganador


## 5. **Cruce y Mutación**
En este paso, se combinan dos cromosomas padres para generar dos hijos. Luego, se aplica una mutación aleatoria que invierte un bit seleccionado al azar en el cromosoma.

In [ ]:
def cruce(padre1, padre2):
    punto_cruce = np.random.randint(1, len(padre1) - 1)
    hijo1 = np.concatenate((padre1[:punto_cruce], padre2[punto_cruce:]))
    hijo2 = np.concatenate((padre2[:punto_cruce], padre1[punto_cruce:]))
    return hijo1, hijo2

def mutacion_un_bit(individuo):
    bit_a_mutar = np.random.randint(len(individuo))
    individuo[bit_a_mutar] = 1 - individuo[bit_a_mutar]
    return individuo


## 6. **Reemplazo Elitista**
El reemplazo elitista asegura que los mejores individuos de la población actual se preserven para la siguiente generación.

In [ ]:
def reemplazo_elitista(poblacion, nueva_poblacion, num_elites):
    poblacion_ordenada = sorted(poblacion, key=lambda ind: funcion_aptitud(*decodificar_cromosoma(ind, 0, 5)), reverse=True)
    nueva_poblacion_ordenada = sorted(nueva_poblacion, key=lambda ind: funcion_aptitud(*decodificar_cromosoma(ind, 0, 5)), reverse=True)
    elitistas = poblacion_ordenada[:num_elites]
    nueva_poblacion_reemplazada = elitistas + nueva_poblacion_ordenada[:len(poblacion) - num_elites]
    return np.array(nueva_poblacion_reemplazada)

## 7. **Ejecución del Algoritmo Genético**
Finalmente, el algoritmo genético se ejecuta iterativamente, creando nuevas generaciones de cromosomas hasta que se alcanza el número deseado de generaciones.

In [ ]:
def algoritmo_genetico(poblacion_inicial, generaciones, k, num_elites):
    poblacion = poblacion_inicial
    historial = []
    for _ in range(generaciones):
        nueva_poblacion = []
        while len(nueva_poblacion) < len(poblacion):
            padre1 = seleccion_torneo(poblacion, k)
            padre2 = seleccion_torneo(poblacion, k)
            hijo1, hijo2 = cruce(padre1, padre2)
            nueva_poblacion.append(mutacion_un_bit(hijo1))
            nueva_poblacion.append(mutacion_un_bit(hijo2))

        # Aplicar reemplazo elitista
        poblacion = reemplazo_elitista(poblacion, nueva_poblacion, num_elites)

    return max(poblacion, key=lambda ind: funcion_aptitud(*decodificar_cromosoma(ind, 0, 5)))




### Ejecución y Resultados
El script principal ejecuta el algoritmo genético con los parámetros definidos y muestra el mejor cromosoma encontrado, junto con sus dimensiones y su aptitud (volumen).


In [ ]:
if __name__ == "__main__":
    n = 20  # Tamaño de la población
    l = 20  # Longitud de cada cromosoma
    generaciones = 100
    k = 2  # Tamaño del torneo
    num_elites = 2  # Número de individuos a preservar

    # Generar población inicial
    poblacion_inicial = generar_poblacion(n, l)

    # Ejecutar algoritmo genético
    mejor_cromosoma = algoritmo_genetico(poblacion_inicial, generaciones, k, num_elites)
    l, w, h = decodificar_cromosoma(mejor_cromosoma, 0, 5)
    print("Mejor solución encontrada:")
    print("Cromosoma:", mejor_cromosoma)
    print("Dimensiones (l, w, h):", [l, w, h])
    print("Aptitud (volumen):", funcion_aptitud(l, w, h))

Mejor solución encontrada:
Cromosoma: [0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0]
Dimensiones (l, w, h): [np.float64(1.1111111111111112), np.float64(2.5396825396825395), np.float64(1.9607843137254901)]
Aptitud (volumen): 5.533077428502264
